In [105]:
# Instalar 1 vez
# !pip install geopy
# !pip install geocoder
# !pip install requests
# !pip install selenium
# !pip install folium

In [106]:
# Cargamos algunas librerías necesarias.
import numpy  as np  
import pandas as pd
pd.set_option('display.max_columns', 100) # Con esto garantizo el ver todas las columnas.

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt # Para dibujar alguna gráfica.
%matplotlib inline

from geopy.geocoders import Nominatim
import geocoder
import requests
import re
import threading 
import time

In [107]:
df = pd.read_csv('./Accidentalidad_final.csv',sep=';', decimal='.', encoding= 'unicode_escape')
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,NÂº,NÂº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,NÂº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,25/01/2017,DE 14:00 A 14:59,MIERCOLES,SAN BLAS,CALLE DE TAMPICO NUM ...,95,2017/799,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÃN DOBLE,NO ASIGNADO,TESTIGO,HOMBRE,IL,DESCONOCIDA
1,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM. ...,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÃN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÃOS
2,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÃN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÃOS
3,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA ...,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÃN DOBLE,TURISMO,CONDUCTOR,HOMBRE,NO ASIGNADA,DE 18 A 20 AÃOS
4,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR ...,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÃOS


In [108]:
df = df.rename(columns={'NÂº PARTE':'Nº PARTE',
                                   'NÂº':'Nº', 'NÂº VICTIMAS': 'Nº VICTIMAS'})

In [109]:
# Quitamos los espacios en blanco porque en Testigo da problema para quitarlos al tener esos espacios.
df['TIPO PERSONA'] = df['TIPO PERSONA'].str.strip()
print (df['TIPO PERSONA'])

0           TESTIGO
1         CONDUCTOR
2           VIAJERO
3         CONDUCTOR
4            PEATON
            ...    
252993    CONDUCTOR
252994    CONDUCTOR
252995    CONDUCTOR
252996      VIAJERO
252997      VIAJERO
Name: TIPO PERSONA, Length: 252998, dtype: object


In [110]:
# Quitamos a los testigos y reiniciamos el índice
df = df[df['TIPO PERSONA'] != 'TESTIGO'].reset_index(drop=True)
df.shape

(221332, 26)

### Unificamos por número de parte

#### Empezamos por TIPO PERSONA

In [111]:
# Agregamos un espacio ahora para la concatenación
df['TIPO PERSONA'] = df['TIPO PERSONA'] + ' | '
df2 = df.groupby(['Nº PARTE'],as_index=False).agg({'TIPO PERSONA': 'sum'})
df2.head()

,Nº PARTE,TIPO PERSONA
0,2010/1000,CONDUCTOR | CONDUCTOR |
1,2010/10000,CONDUCTOR | CONDUCTOR |
2,2010/10001,VIAJERO | PEATON | CONDUCTOR | VIAJERO |
3,2010/10003,CONDUCTOR | CONDUCTOR | CONDUCTOR | VIAJERO | ...
4,2010/10004,CONDUCTOR | CONDUCTOR |


#### Seguimos con TIPO ACCIDENTE

In [112]:
# Quitamos los espacios en blanco
df['TIPO ACCIDENTE'] = df['TIPO ACCIDENTE'].str.strip()
df['TIPO ACCIDENTE']

0             COLISIÃN DOBLE
1             COLISIÃN DOBLE
2             COLISIÃN DOBLE
3                   ATROPELLO
4             COLISIÃN DOBLE
                 ...         
221327    COLISIÃN MÃLTIPLE
221328        COLISIÃN DOBLE
221329        COLISIÃN DOBLE
221330        COLISIÃN DOBLE
221331              ATROPELLO
Name: TIPO ACCIDENTE, Length: 221332, dtype: object

In [113]:
# Agregamos un espacio ahora para la concatenación
df['TIPO ACCIDENTE'] = df['TIPO ACCIDENTE'] + ' | '
df3 = df.groupby(['Nº PARTE'],as_index=False).agg({'TIPO ACCIDENTE': 'sum'})
df3.head()

,Nº PARTE,TIPO ACCIDENTE
0,2010/1000,COLISIÃN DOBLE | COLISIÃN DOBLE |
1,2010/10000,COLISIÃN DOBLE | COLISIÃN DOBLE |
2,2010/10001,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO |
3,2010/10003,COLISIÃN MÃLTIPLE | COLISIÃN MÃLTIPLE | CO...
4,2010/10004,COLISIÃN DOBLE | COLISIÃN DOBLE |


#### Continuamos con Tipo Vehiculo

In [114]:
# Quitamos los espacios en blanco
df['Tipo Vehiculo'] = df['Tipo Vehiculo'].str.strip()
df['Tipo Vehiculo']

0             MOTOCICLETA
1                 TURISMO
2                 TURISMO
3             NO ASIGNADO
4             MOTOCICLETA
               ...       
221327            TURISMO
221328    AUTOBUS-AUTOCAR
221329          AUTO-TAXI
221330            TURISMO
221331            TURISMO
Name: Tipo Vehiculo, Length: 221332, dtype: object

In [115]:
# Agregamos un espacio ahora para la concatenación
df['Tipo Vehiculo'] = df['Tipo Vehiculo'] + ' | '
df4 = df.groupby(['Nº PARTE'],as_index=False).agg({'Tipo Vehiculo': 'sum'})
df4.head()

,Nº PARTE,Tipo Vehiculo
0,2010/1000,CICLOMOTOR | TURISMO |
1,2010/10000,MOTOCICLETA | TURISMO |
2,2010/10001,TURISMO | NO ASIGNADO | TURISMO | TURISMO |
3,2010/10003,TURISMO | FURGONETA | TURISMO | AUTOBUS-AUTOCA...
4,2010/10004,TURISMO | MOTOCICLETA |


#### Continuamos con SEXO

In [116]:
# Quitamos los espacios en blanco
df['SEXO'] = df['SEXO'].str.strip()
df['SEXO']

0         HOMBRE
1          MUJER
2         HOMBRE
3         HOMBRE
4         HOMBRE
           ...  
221327     MUJER
221328    HOMBRE
221329    HOMBRE
221330    HOMBRE
221331    HOMBRE
Name: SEXO, Length: 221332, dtype: object

In [117]:
# Agregamos un espacio ahora para la concatenación
df['SEXO'] = df['SEXO'] + ' | '
df5 = df.groupby(['Nº PARTE'],as_index=False).agg({'SEXO': 'sum'})
df5.head()

,Nº PARTE,SEXO
0,2010/1000,HOMBRE | HOMBRE |
1,2010/10000,HOMBRE | HOMBRE |
2,2010/10001,HOMBRE | MUJER | HOMBRE | HOMBRE |
3,2010/10003,HOMBRE | HOMBRE | HOMBRE | HOMBRE | HOMBRE |
4,2010/10004,MUJER | HOMBRE |


#### Continuamos con LESIVIDAD

In [118]:
# Quitamos los espacios en blanco
df['LESIVIDAD'] = df['LESIVIDAD'].str.strip()
df['LESIVIDAD']

0                  HL
1                  HL
2         NO ASIGNADA
3                  HL
4                  HL
             ...     
221327    NO ASIGNADA
221328             IL
221329             IL
221330             IL
221331             IL
Name: LESIVIDAD, Length: 221332, dtype: object

In [119]:
# Agregamos un espacio ahora para la concatenación
df['LESIVIDAD'] = df['LESIVIDAD'] + ' | '
df6 = df.groupby(['Nº PARTE'],as_index=False).agg({'LESIVIDAD': 'sum'})
df6.head()

,Nº PARTE,LESIVIDAD
0,2010/1000,HL | IL |
1,2010/10000,HG | IL |
2,2010/10001,IL | HG | IL | IL |
3,2010/10003,IL | IL | HL | HL | IL |
4,2010/10004,IL | HL |


#### Y finalizamos con Tramo Edad

In [120]:
# Quitamos los espacios en blanco
df['Tramo Edad'] = df['Tramo Edad'].str.strip()
df['Tramo Edad']

0         DE 21 A 24 AÃOS
1         DE 18 A 20 AÃOS
2         DE 18 A 20 AÃOS
3         DE 35 A 39 AÃOS
4          DE 30 A 34 ANOS
                ...       
221327     DE 30 A 34 ANOS
221328    DE 55 A 59 AÃOS
221329    DE 45 A 49 AÃOS
221330    DE 50 A 54 AÃOS
221331         DESCONOCIDA
Name: Tramo Edad, Length: 221332, dtype: object

In [121]:
# Agregamos un espacio ahora para la concatenación
df['Tramo Edad'] = df['Tramo Edad'] + ' | '
df7 = df.groupby(['Nº PARTE'],as_index=False).agg({'Tramo Edad': 'sum'})
df7.head()

,Nº PARTE,Tramo Edad
0,2010/1000,DE 25 A 29 AÃOS | DE 35 A 39 AÃOS |
1,2010/10000,DE 35 A 39 AÃOS | DE 45 A 49 AÃOS |
2,2010/10001,DE 6 A 9 AÃOS | DE 45 A 49 AÃOS | DE 55 A 59...
3,2010/10003,DE 50 A 54 AÃOS | DE 65 A 69 AÃOS | DE 21 A ...
4,2010/10004,DE 40 A 44 AÃOS | DE 40 A 44 AÃOS |


### Ahora quitamos los partes duplicados y concatenamos con los df que hemos creado anteriormente

In [122]:
# Nos quedamos con 1 único parte
df = df.drop_duplicates(subset=['Nº PARTE'])
df.shape

(86817, 26)

In [123]:
# Borramos nuestras antiguas columnas para sustituirlas por las nuevas que hemos creado
df.drop(['TIPO ACCIDENTE', 'Tipo Vehiculo', 'TIPO PERSONA', 'SEXO', 'LESIVIDAD', 'Tramo Edad'], axis=1, inplace=True)

In [124]:
df = df.join(df2.set_index('Nº PARTE'), on='Nº PARTE')
df = df.join(df3.set_index('Nº PARTE'), on='Nº PARTE')
df = df.join(df4.set_index('Nº PARTE'), on='Nº PARTE')
df = df.join(df5.set_index('Nº PARTE'), on='Nº PARTE')
df = df.join(df6.set_index('Nº PARTE'), on='Nº PARTE')
df = df.join(df7.set_index('Nº PARTE'), on='Nº PARTE')

In [125]:
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM. ...,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA ...,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR ...,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO ...,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...


In [126]:
# Reseteamos el índice nuevamente
df = df.reset_index(drop=True)
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM. ...,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA ...,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR ...,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO ...,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...


## Coordenadas Calles con Número

In [127]:
stop_words = ['num', 'NUM']

stop_words = set(stop_words)
f = lambda x: ' '.join(w for w in x.split() if not w in stop_words)
df['LUGAR ACCIDENTE'] = df['LUGAR ACCIDENTE'].apply(f)

In [128]:
df['LUGAR ACCIDENTE'][13]

'CALLE DE LA VIRGEN DE AFRICA'

In [129]:
df['Calle completa'] = df['LUGAR ACCIDENTE'] +' '+ df['Nº'] + ', Madrid, Spain'
df['Calle completa'].head()

0      AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain
1    AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...
2    PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...
3    CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...
4    CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...
Name: Calle completa, dtype: object

In [130]:
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain"
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad..."
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ..."
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ..."


In [131]:
df['Calle completa'][13]

'CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain'

In [132]:
df2 = df

In [133]:
df2.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain"
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad..."
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ..."
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ..."


In [134]:
# Quitamos los cruces
df2 = df2[~df2['Calle completa'].str.contains("-")]
df2

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa
7,16/08/2011,DE 12:00 A 12:59,MARTES,CENTRO,CALLE DEL ARENAL,23,2011/7464,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | PEATON |,ATROPELLO | ATROPELLO |,TURISMO | NO ASIGNADO |,HOMBRE | MUJER |,IL | HL |,DE MAS DE 74 AÃOS | DE 18 A 20 AÃOS |,"CALLE DEL ARENAL 23, Madrid, Spain"
11,27/01/2018,DE 5:00 A 5:59,SABADO,SALAMANCA,CALLE DE JORGE JUAN,1,2018/1026,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | CAMION |,HOMBRE | HOMBRE |,HG | IL |,DE 30 A 34 ANOS | DE 60 A 64 AÃOS |,"CALLE DE JORGE JUAN 1, Madrid, Spain"
12,23/12/2011,DE 13:00 A 13:59,VIERNES,TETUAN,PASEO DE LA CASTELLANA,101,2011/12187,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,VIAJERO | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | MOTOCICLETA |,MUJER | MUJER | HOMBRE |,IL | IL | HL |,DE 21 A 24 AÃOS | DE 35 A 39 AÃOS | DE 30 A ...,"PASEO DE LA CASTELLANA 101, Madrid, Spain"
13,22/12/2017,DE 15:00 A 15:59,VIERNES,CIUDAD LINEAL,CALLE DE LA VIRGEN DE AFRICA,6,2017/12631,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,IL | HL | IL |,DE 40 A 44 AÃOS | DE 40 A 44 AÃOS | DE 35 A ...,"CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain"
15,21/06/2016,DE 19:00 A 19:59,MARTES,CIUDAD LINEAL,AVENIDA DE AMERICA KM.,4500,2016/5990,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,2.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | VIAJERO | ...,COLISIÃN MÃLTIPLE | COLISIÃN MÃLTIPLE | CO...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | MUJER | HOMBRE | MUJER | MUJER |,HL | IL | IL | IL | HL |,DE 40 A 44 AÃOS | DE 40 A 44 AÃOS | DE 30 A ...,"AVENIDA DE AMERICA KM. 4500, Madrid, Spain"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86806,15/06/2011,DE 21:00 A 21:59,MIERCOLES,ARGANZUELA,PLAZA DE LEGAZPI,1,2011/5586,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 40 A 44 AÃOS |,"PLAZA DE LEGAZPI 1, Madrid, Spain"
86807,03/06/2016,DE 11:00 A 11:59,VIERNES,CHAMBERI,CALLE DE SAGASTA,11,2016/5323,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 60 A 64 AÃOS |,"CALLE DE SAGASTA 11, Madrid, Spain"
86810,05/09/2018,DE 23:00 A 23:59,MIERCOLES,CHAMARTIN,AVENIDA DE AMERICA ENTRADA KM.,4,2018/9585,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 25 A 29 AÃOS |,"AVENIDA DE AMERICA ENTRADA KM. 4, Madrid, Spain"
86815,29/05/2015,DE 21:00 A 21:59,VIERNES,SALAMANCA,CALLE DEL ALCALDE SAINZ DE BARANDA,92,2015/5089,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 30 A 34 ANOS |,"CALLE DEL ALCALDE SAINZ DE BARANDA 92, Madrid,..."


In [135]:
# Quitamos la M-30 que también da problemas con las coordenadas
df2 = df2[~df2['Calle completa'].str.contains("AUTOVIA")]
df2

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa
7,16/08/2011,DE 12:00 A 12:59,MARTES,CENTRO,CALLE DEL ARENAL,23,2011/7464,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | PEATON |,ATROPELLO | ATROPELLO |,TURISMO | NO ASIGNADO |,HOMBRE | MUJER |,IL | HL |,DE MAS DE 74 AÃOS | DE 18 A 20 AÃOS |,"CALLE DEL ARENAL 23, Madrid, Spain"
11,27/01/2018,DE 5:00 A 5:59,SABADO,SALAMANCA,CALLE DE JORGE JUAN,1,2018/1026,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | CAMION |,HOMBRE | HOMBRE |,HG | IL |,DE 30 A 34 ANOS | DE 60 A 64 AÃOS |,"CALLE DE JORGE JUAN 1, Madrid, Spain"
12,23/12/2011,DE 13:00 A 13:59,VIERNES,TETUAN,PASEO DE LA CASTELLANA,101,2011/12187,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,VIAJERO | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | MOTOCICLETA |,MUJER | MUJER | HOMBRE |,IL | IL | HL |,DE 21 A 24 AÃOS | DE 35 A 39 AÃOS | DE 30 A ...,"PASEO DE LA CASTELLANA 101, Madrid, Spain"
13,22/12/2017,DE 15:00 A 15:59,VIERNES,CIUDAD LINEAL,CALLE DE LA VIRGEN DE AFRICA,6,2017/12631,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,IL | HL | IL |,DE 40 A 44 AÃOS | DE 40 A 44 AÃOS | DE 35 A ...,"CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain"
15,21/06/2016,DE 19:00 A 19:59,MARTES,CIUDAD LINEAL,AVENIDA DE AMERICA KM.,4500,2016/5990,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,2.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | VIAJERO | ...,COLISIÃN MÃLTIPLE | COLISIÃN MÃLTIPLE | CO...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | MUJER | HOMBRE | MUJER | MUJER |,HL | IL | IL | IL | HL |,DE 40 A 44 AÃOS | DE 40 A 44 AÃOS | DE 30 A ...,"AVENIDA DE AMERICA KM. 4500, Madrid, Spain"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86806,15/06/2011,DE 21:00 A 21:59,MIERCOLES,ARGANZUELA,PLAZA DE LEGAZPI,1,2011/5586,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 40 A 44 AÃOS |,"PLAZA DE LEGAZPI 1, Madrid, Spain"
86807,03/06/2016,DE 11:00 A 11:59,VIERNES,CHAMBERI,CALLE DE SAGASTA,11,2016/5323,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 60 A 64 AÃOS |,"CALLE DE SAGASTA 11, Madrid, Spain"
86810,05/09/2018,DE 23:00 A 23:59,MIERCOLES,CHAMARTIN,AVENIDA DE AMERICA ENTRADA KM.,4,2018/9585,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 25 A 29 AÃOS |,"AVENIDA DE AMERICA ENTRADA KM. 4, Madrid, Spain"
86815,29/05/2015,DE 21:00 A 21:59,VIERNES,SALAMANCA,CALLE DEL ALCALDE SAINZ DE BARANDA,92,2015/5089,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 30 A 34 ANOS |,"CALLE DEL ALCALDE SAINZ DE BARANDA 92, Madrid,..."


In [136]:
len(df2['Calle completa'].unique())

18927

In [137]:
df2['Calle completa'].unique()

array(['CALLE DEL ARENAL 23, Madrid, Spain',
       'CALLE DE JORGE JUAN 1, Madrid, Spain',
       'PASEO DE LA CASTELLANA 101, Madrid, Spain', ...,
       'CALLE DE LOS ALGODONALES 54, Madrid, Spain',
       'CALLE 30 NC KM. 14800, Madrid, Spain',
       'CALLE DE SAGASTA 11, Madrid, Spain'], dtype=object)

In [138]:
dfCC = pd.DataFrame(df2['Calle completa'].unique()).astype(str)

In [139]:
dfCC['Calle completa'] = pd.DataFrame(df2['Calle completa'].unique()).astype(str)

In [140]:
dfCC

,0,Calle completa
0,"CALLE DEL ARENAL 23, Madrid, Spain","CALLE DEL ARENAL 23, Madrid, Spain"
1,"CALLE DE JORGE JUAN 1, Madrid, Spain","CALLE DE JORGE JUAN 1, Madrid, Spain"
2,"PASEO DE LA CASTELLANA 101, Madrid, Spain","PASEO DE LA CASTELLANA 101, Madrid, Spain"
3,"CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain","CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain"
4,"AVENIDA DE AMERICA KM. 4500, Madrid, Spain","AVENIDA DE AMERICA KM. 4500, Madrid, Spain"
...,...,...
18922,"PLAZA DE BARBADOS 0, Madrid, Spain","PLAZA DE BARBADOS 0, Madrid, Spain"
18923,"CALLE DE EMBAJADORES 117, Madrid, Spain","CALLE DE EMBAJADORES 117, Madrid, Spain"
18924,"CALLE DE LOS ALGODONALES 54, Madrid, Spain","CALLE DE LOS ALGODONALES 54, Madrid, Spain"
18925,"CALLE 30 NC KM. 14800, Madrid, Spain","CALLE 30 NC KM. 14800, Madrid, Spain"


In [141]:
# from tqdm import tqdm
# tqdm.pandas()
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="TheCrash")

# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# dfCC['location'] = dfCC['Calle Completa'].progress_apply(geocode)

# dfCC['point'] = dfCC['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [142]:
# dfCC.to_csv('./df32018_Accidentalidad.csv',sep=';', decimal='.')

In [143]:
dfCC = pd.read_csv('./df32018_Accidentalidad.csv',sep=';', decimal='.')
dfCC.head()

,Unnamed: 0,0,Calle Completa,location,point
0,0,"CALLE DEL ARENAL 23, Madrid, Spain","CALLE DEL ARENAL 23, Madrid, Spain","Calle del Arenal, Húmera, Pozuelo de Alarcón, ...","(40.4292084, -3.7846342, 0.0)"
1,1,"CALLE DE JORGE JUAN 1, Madrid, Spain","CALLE DE JORGE JUAN 1, Madrid, Spain","Calle de Jorge Juan, Recoletos, Salamanca, Mad...","(40.4236967, -3.682893, 0.0)"
2,2,"PASEO DE LA CASTELLANA 101, Madrid, Spain","PASEO DE LA CASTELLANA 101, Madrid, Spain","Paseo de la Castellana, Ríos Rosas, Chamberí, ...","(40.44142, -3.6919575, 0.0)"
3,3,"CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain","CALLE DE LA VIRGEN DE AFRICA 6, Madrid, Spain","Calle de la Virgen de África, San Pascual, Ciu...","(40.4387705, -3.6544403, 0.0)"
4,4,"AVENIDA DE AMERICA KM. 4500, Madrid, Spain","AVENIDA DE AMERICA KM. 4500, Madrid, Spain","Avenida de América, Parque de Lisboa, El Praon...","(40.35871775, -3.8203344946376587, 0.0)"


In [144]:
dfCC['point'].value_counts().sum()

16329

In [145]:
dfCC['point'].isnull().sum()

2598

In [146]:
dfCC.dtypes

Unnamed: 0         int64
0                 object
Calle Completa    object
location          object
point             object
dtype: object

In [147]:
df['point'] = df['Calle completa'].map(dfCC.set_index('Calle Completa')['point'])

In [148]:
df

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain",NaN
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,NaN
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...",NaN
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...",NaN
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,13/10/2012,DE 1:00 A 1:59,SABADO,FUENCARRAL-EL PARDO,PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR,0,2012/9229,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 25 A 29 AÃOS |,PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR...,NaN
86813,21/05/2015,DE 9:00 A 9:59,JUEVES,CHAMBERI,PASEO DE LA CASTELLANA - PLAZA DE COLON,0,2015/4635,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,MUJER |,HL |,DE 25 A 29 AÃOS |,"PASEO DE LA CASTELLANA - PLAZA DE COLON 0, Mad...",NaN
86814,29/12/2015,DE 5:00 A 5:59,MARTES,MONCLOA-ARAVACA,AUTOVIA M-30 CALZADA 1 KM.,23000,2015/11845,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,VUELCO |,TURISMO |,HOMBRE |,HL |,DE 40 A 44 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 23000, Madrid, Spain",NaN
86815,29/05/2015,DE 21:00 A 21:59,VIERNES,SALAMANCA,CALLE DEL ALCALDE SAINZ DE BARANDA,92,2015/5089,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 30 A 34 ANOS |,"CALLE DEL ALCALDE SAINZ DE BARANDA 92, Madrid,...","(40.4173348, -3.6675712, 0.0)"


### Ahora con los cruces

In [149]:
df['Calle completa']

0          AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain
1        AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...
2        PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...
3        CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...
4        CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...
                               ...                        
86812    PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR...
86813    PASEO DE LA CASTELLANA - PLAZA DE COLON 0, Mad...
86814      AUTOVIA M-30 CALZADA 1 KM. 23000, Madrid, Spain
86815    CALLE DEL ALCALDE SAINZ DE BARANDA 92, Madrid,...
86816              CALLE DE JOSE ABASCAL 56, Madrid, Spain
Name: Calle completa, Length: 86817, dtype: object

In [150]:
# Vemos que hay unos 40000 cruces en el dataset
len(df[df['Calle completa'].str.contains(' - ')])

40332

In [151]:
df['CruceCalle'] = df['LUGAR ACCIDENTE'][df['LUGAR ACCIDENTE'].str.contains(" - ")].str.split("-")

In [152]:
df3 = df.CruceCalle.apply(pd.Series)

In [153]:
df3.head()

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN
1,AVENIDA DE LA GRAN VIA DEL ESTE,CALLE DE LA PIROTECNIA,NaN,NaN,NaN
2,PASEO DE LAS DELICIAS,CALLE DE MURCIA,NaN,NaN,NaN
3,CALLE DE EROS,CALLE DE LA ESTRELLA HADAR,NaN,NaN,NaN
4,CALLE DE BELZUNEGUI,CALLE DE CARCASTILLO,NaN,NaN,NaN


In [154]:
df3.columns = ['Calle1','Calle2','nada1', 'nada2', 'nada3']
df3.drop(['nada1', 'nada2', 'nada3'], axis=1, inplace=True)
df3.head()

,Calle1,Calle2
0,NaN,NaN
1,AVENIDA DE LA GRAN VIA DEL ESTE,CALLE DE LA PIROTECNIA
2,PASEO DE LAS DELICIAS,CALLE DE MURCIA
3,CALLE DE EROS,CALLE DE LA ESTRELLA HADAR
4,CALLE DE BELZUNEGUI,CALLE DE CARCASTILLO


In [155]:
# Quitamos NaN
df3 = df3.dropna()

In [156]:
df3

,Calle1,Calle2
1,AVENIDA DE LA GRAN VIA DEL ESTE,CALLE DE LA PIROTECNIA
2,PASEO DE LAS DELICIAS,CALLE DE MURCIA
3,CALLE DE EROS,CALLE DE LA ESTRELLA HADAR
4,CALLE DE BELZUNEGUI,CALLE DE CARCASTILLO
5,AVENIDA DEL DOCTOR GARCIA TAPIA,AUTOVIA M
...,...,...
86808,CALLE DEL CONVENIO,AUTOVIA M
86809,CALLE DE COSTA RICA,CALLE DE PUERTO RICO
86811,CARRETERA A LA ESTACION DE HORTALEZA,CALLE DE SANTA ADELA
86812,PASEO DE LA CASTELLANA,CARRETERA DE COLMENAR


In [157]:
# Le agregamos "Madrid, Spain" para facilitar la búsqueda.
df3['Calle1'] = df3['Calle1'] + ', Madrid, Spain'
df3['Calle1']

1          AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain
2                    PASEO DE LAS DELICIAS , Madrid, Spain
3                            CALLE DE EROS , Madrid, Spain
4                      CALLE DE BELZUNEGUI , Madrid, Spain
5          AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain
                               ...                        
86808                   CALLE DEL CONVENIO , Madrid, Spain
86809                  CALLE DE COSTA RICA , Madrid, Spain
86811    CARRETERA A LA ESTACION DE HORTALEZA , Madrid,...
86812               PASEO DE LA CASTELLANA , Madrid, Spain
86813               PASEO DE LA CASTELLANA , Madrid, Spain
Name: Calle1, Length: 40294, dtype: object

In [158]:
len(df3['Calle1'].unique())

3835

In [159]:
dfCr = pd.DataFrame(df3['Calle1'].unique()).astype(str)

In [160]:
dfCr['Calle1'] = pd.DataFrame(df3['Calle1'].unique()).astype(str)
dfCr

,0,Calle1
0,"AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain"
1,"PASEO DE LAS DELICIAS , Madrid, Spain","PASEO DE LAS DELICIAS , Madrid, Spain"
2,"CALLE DE EROS , Madrid, Spain","CALLE DE EROS , Madrid, Spain"
3,"CALLE DE BELZUNEGUI , Madrid, Spain","CALLE DE BELZUNEGUI , Madrid, Spain"
4,"AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain","AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain"
...,...,...
3830,"CALLE PRINCIPAL DE PROVINCIAS , Madrid, Spain","CALLE PRINCIPAL DE PROVINCIAS , Madrid, Spain"
3831,"CALLE DE HISTORIAS DE LA RADIO , Madrid, Spain","CALLE DE HISTORIAS DE LA RADIO , Madrid, Spain"
3832,"CALLE DE JOSE NORIEGA , Madrid, Spain","CALLE DE JOSE NORIEGA , Madrid, Spain"
3833,"CALLE DEL RASTRO , Madrid, Spain","CALLE DEL RASTRO , Madrid, Spain"


In [161]:
# from tqdm import tqdm
# tqdm.pandas()
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="TheCrash")

# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# dfCr['location'] = dfCr['Calle1'].progress_apply(geocode)

# dfCr['point'] = dfCr['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [162]:
# dfCr.to_csv('./df42018_Accidentalidad.csv',sep=';', decimal='.')

In [163]:
dfCr = pd.read_csv('./df42018_Accidentalidad.csv',sep=';', decimal='.')
dfCr.head()

,Unnamed: 0,0,Calle1,location,point
0,0,"AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","Avenida de la Gran Vía del Este, Valderribas, ...","(40.4031169, -3.595555, 0.0)"
1,1,"PASEO DE LAS DELICIAS , Madrid, Spain","PASEO DE LAS DELICIAS , Madrid, Spain","Paseo de las Delicias, Delicias, Arganzuela, M...","(40.3945911, -3.6946377, 0.0)"
2,2,"CALLE DE EROS , Madrid, Spain","CALLE DE EROS , Madrid, Spain","Calle de Eros, Legazpi, Arganzuela, Madrid, Ár...","(40.3938219, -3.6802947, 0.0)"
3,3,"CALLE DE BELZUNEGUI , Madrid, Spain","CALLE DE BELZUNEGUI , Madrid, Spain","Calle de Belzunegui, Colonia de la Prensa, Car...","(40.375867, -3.739077, 0.0)"
4,4,"AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain","AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain","Avenida del Doctor García Tapia, Marroquina, M...","(40.4129832, -3.6559716, 0.0)"


In [164]:
df['Calle1'] = df3['Calle1']

In [165]:
df['point2'] = df['Calle1'].map(dfCr.set_index('Calle1')['point'])

In [166]:
df

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point,CruceCalle,Calle1,point2
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain",NaN,NaN,NaN,NaN
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,NaN,"[AVENIDA DE LA GRAN VIA DEL ESTE , CALLE DE L...","AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","(40.4031169, -3.595555, 0.0)"
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...",NaN,"[PASEO DE LAS DELICIAS , CALLE DE MURCIA]","PASEO DE LAS DELICIAS , Madrid, Spain","(40.3945911, -3.6946377, 0.0)"
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...",NaN,"[CALLE DE EROS , CALLE DE LA ESTRELLA HADAR]","CALLE DE EROS , Madrid, Spain","(40.3938219, -3.6802947, 0.0)"
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...",NaN,"[CALLE DE BELZUNEGUI , CALLE DE CARCASTILLO]","CALLE DE BELZUNEGUI , Madrid, Spain","(40.375867, -3.739077, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,13/10/2012,DE 1:00 A 1:59,SABADO,FUENCARRAL-EL PARDO,PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR,0,2012/9229,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 25 A 29 AÃOS |,PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR...,NaN,"[PASEO DE LA CASTELLANA , CARRETERA DE COLMENAR]","PASEO DE LA CASTELLANA , Madrid, Spain","(40.4491807, -3.6910059, 0.0)"
86813,21/05/2015,DE 9:00 A 9:59,JUEVES,CHAMBERI,PASEO DE LA CASTELLANA - PLAZA DE COLON,0,2015/4635,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,MUJER |,HL |,DE 25 A 29 AÃOS |,"PASEO DE LA CASTELLANA - PLAZA DE COLON 0, Mad...",NaN,"[PASEO DE LA CASTELLANA , PLAZA DE COLON]","PASEO DE LA CASTELLANA , Madrid, Spain","(40.4491807, -3.6910059, 0.0)"
86814,29/12/2015,DE 5:00 A 5:59,MARTES,MONCLOA-ARAVACA,AUTOVIA M-30 CALZAD

In [167]:
df['point'] = df['point'].fillna(df['point2'])

In [168]:
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point,CruceCalle,Calle1,point2
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain",NaN,NaN,NaN,NaN
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,"(40.4031169, -3.595555, 0.0)","[AVENIDA DE LA GRAN VIA DEL ESTE , CALLE DE L...","AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","(40.4031169, -3.595555, 0.0)"
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...","(40.3945911, -3.6946377, 0.0)","[PASEO DE LAS DELICIAS , CALLE DE MURCIA]","PASEO DE LAS DELICIAS , Madrid, Spain","(40.3945911, -3.6946377, 0.0)"
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...","(40.3938219, -3.6802947, 0.0)","[CALLE DE EROS , CALLE DE LA ESTRELLA HADAR]","CALLE DE EROS , Madrid, Spain","(40.3938219, -3.6802947, 0.0)"
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...","(40.375867, -3.739077, 0.0)","[CALLE DE BELZUNEGUI , CALLE DE CARCASTILLO]","CALLE DE BELZUNEGUI , Madrid, Spain","(40.375867, -3.739077, 0.0)"
5,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,CONDUCTOR | VIAJERO | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,FURGONETA | FURGONETA | FURGONETA | TURISMO | ...,HOMBRE | MUJER | MUJER | MUJER | MUJER |,IL | IL | HL | HL | HL |,DE 21 A 24 AÃOS | DE 0 A 5 AÃOS | DE 21 A 24...,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40...,"(40.4129832, -3.6559716, 0.0)","[AVENIDA DEL DOCTOR GARCIA TAPIA , AUTOVIA M,...","AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain","(40.4129832, -3.6559716, 0.0)"
6,31/05/2016,DE 13:00 A 13:59,MARTES,TETUAN,AVENIDA DEL GENERAL PERON - CALLE DEL CAPITAN ...,0,2016/5276,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | MUJER |,HG | IL |,DE 35 A 39 AÃ

In [169]:
df['point'].isnull().sum()

14227

### Finalmente hacemos las coordenadas de la M30

In [307]:
# Hay unas 6500 filas de la M30 y otras autovías.
len(df[df['Calle completa'].str.contains('AUTOVIA')])

6516

In [308]:
df[df['Calle completa'].str.contains('AUTOVIA')]

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point,CruceCalle,Calle1,point2,Autovia
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 CHAMARTIN , ..."
5,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,CONDUCTOR | VIAJERO | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,FURGONETA | FURGONETA | FURGONETA | TURISMO | ...,HOMBRE | MUJER | MUJER | MUJER | MUJER |,IL | IL | HL | HL | HL |,DE 21 A 24 AÃOS | DE 0 A 5 AÃOS | DE 21 A 24...,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40...,"(40.4129832, -3.6559716, 0.0)","[AVENIDA DEL DOCTOR GARCIA TAPIA , AUTOVIA M,...","AVENIDA DEL DOCTOR GARCIA TAPIA , Madrid, Spain","(40.4129832, -3.6559716, 0.0)",AVENIDA DEL DOCTOR GARCIA TAPIA MORATALAZ ...
10,15/10/2016,DE 18:00 A 18:59,SABADO,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,15000,2016/9662,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,CHOQUE CON OBJETO FIJO | CHOQUE CON OBJETO FIJ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,HOMBRE | MUJER | HOMBRE | HOMBRE | MUJER | MUJ...,HL | IL | HL | IL | HL | IL |,DE 25 A 29 AÃOS | DE 45 A 49 AÃOS | DE 25 A ...,"AUTOVIA M-30 CALZADA 1 KM. 15000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."
19,25/06/2013,DE 14:00 A 14:59,MARTES,MORATALAZ,AUTOVIA M-30 CALZADA 2 KM.,8900,2013/5848,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,TURISMO | TURISMO |,MUJER | MUJER |,HL | IL |,DE 40 A 44 AÃOS | DE 50 A 54 AÃOS |,"AUTOVIA M-30 CALZADA 2 KM. 8900, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 MORATALAZ , ..."
20,11/02/2017,DE 13:00 A 13:59,SABADO,MONCLOA-ARAVACA,AUTOVIA M-30 CALZADA 2 KM.,18100,2017/1441,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,HOMBRE |,HL |,DE 40 A 44 AÃOS |,"AUTOVIA M-30 CALZADA 2 KM. 18100, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 MONCLOA-ARAVACA , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86765,03/12/2010,DE 19:00 A 19:59,VIERNES,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,12000,2010/11139,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 21 A 24 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 12000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."
86769,11/11/2014,DE 10:00 A 10:59,MARTES,FUENCARRAL-EL PARDO,AUTOVIA M-30 CALZADA 2 KM.,32000,2014/10304,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 FUENCARRAL-EL PARDO , ..."
86795,23/05/2010,DE 6:00 A 6:59,DOMINGO,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,14200,2010/4575,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 21 A 24 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 14200, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."
86808,29/05/2012,DE 20:00 A 20:59,MARTES,PUENTE DE VALLECAS,CALLE DEL CONVENIO - AUTOVIA M-30 CALZADA 2,0,2012/5098,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA CICLOMOTOR |,CICLOMOTOR |,HOMBRE |,HL |,DE 35 A 39 AÃOS |,"CALLE DEL CONVENIO - AUTOVIA M-30 CALZADA 2 0,...",NaN,"[CALLE DEL CONVENI

In [309]:
df['Autovia'] = df['LUGAR ACCIDENTE']

In [310]:
df['Autovia'] = df.Autovia.apply(pd.Series)

In [311]:
# df.apply(lambda x: x.str.split(n=2).str[0])

In [312]:
# Quitamos CALZADA y demás palabras que puedan entorpecer la búsqueda.
df['Autovia'] = df['Autovia'].str.replace('CALZADA 1 KM.','')
df['Autovia'] = df['Autovia'].str.replace('CALZADA 2 KM.','')
df['Autovia'] = df['Autovia'].str.replace('CALZADA 2','')
df['Autovia'] = df['Autovia'].str.replace('CALZADA','')
df['Autovia'] = df['Autovia'].str.replace('1','')
df['Autovia'] = df['Autovia'].str.replace(' 2','')
df['Autovia'] = df['Autovia'].str.replace(' - AUTOVIA M-40','')
df['Autovia'] = df['Autovia'].str.replace(' - AUTOVIA M-30','')
df['Autovia'] = df['Autovia'].str.replace('ENTRADA KM.','')
df['Autovia']

0                                            AUTOVIA M-30 
1        AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...
2                  PASEO DE LAS DELICIAS - CALLE DE MURCIA
3               CALLE DE EROS - CALLE DE LA ESTRELLA HADAR
4               CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO
                               ...                        
86812       PASEO DE LA CASTELLANA - CARRETERA DE COLMENAR
86813              PASEO DE LA CASTELLANA - PLAZA DE COLON
86814                                        AUTOVIA M-30 
86815                   CALLE DEL ALCALDE SAINZ DE BARANDA
86816                                CALLE DE JOSE ABASCAL
Name: Autovia, Length: 86817, dtype: object

In [313]:
# Agregamos el distrito para hacer la búsqueda correctamente. Ya que sino no sabe donde ubicarlo.
df['Autovia'] = df['Autovia'] + ' ' + df['DISTRITO'] + ', Madrid, Spain'

In [314]:
# Vemos como quedaría la columna de Autovía.
df[df['Autovia'].str.contains('AUTOVIA')]

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point,CruceCalle,Calle1,point2,Autovia
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 CHAMARTIN , ..."
10,15/10/2016,DE 18:00 A 18:59,SABADO,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,15000,2016/9662,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,CHOQUE CON OBJETO FIJO | CHOQUE CON OBJETO FIJ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,HOMBRE | MUJER | HOMBRE | HOMBRE | MUJER | MUJ...,HL | IL | HL | IL | HL | IL |,DE 25 A 29 AÃOS | DE 45 A 49 AÃOS | DE 25 A ...,"AUTOVIA M-30 CALZADA 1 KM. 15000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."
19,25/06/2013,DE 14:00 A 14:59,MARTES,MORATALAZ,AUTOVIA M-30 CALZADA 2 KM.,8900,2013/5848,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,TURISMO | TURISMO |,MUJER | MUJER |,HL | IL |,DE 40 A 44 AÃOS | DE 50 A 54 AÃOS |,"AUTOVIA M-30 CALZADA 2 KM. 8900, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 MORATALAZ , ..."
20,11/02/2017,DE 13:00 A 13:59,SABADO,MONCLOA-ARAVACA,AUTOVIA M-30 CALZADA 2 KM.,18100,2017/1441,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,HOMBRE |,HL |,DE 40 A 44 AÃOS |,"AUTOVIA M-30 CALZADA 2 KM. 18100, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 MONCLOA-ARAVACA , ..."
22,26/03/2012,DE 9:00 A 9:59,LUNES,MORATALAZ,AUTOVIA M-23 ENTRADA KM.,0,2012/2828,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,CONDUCTOR | CONDUCTOR | VIAJERO | CONDUCTOR | ...,COLISIÃN MÃLTIPLE | COLISIÃN MÃLTIPLE | CO...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | MUJER | NO ASIGNADO | HOMBRE ...,HL | IL | HL | NO ASIGNADA | HL | IL | IL |,DE 30 A 34 ANOS | DE 35 A 39 AÃOS | DE 30 A 3...,"AUTOVIA M-23 ENTRADA KM. 0, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-23 MORATALAZ , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86758,19/10/2016,DE 16:00 A 16:59,MIERCOLES,SALAMANCA,AUTOVIA M-30 CALZADA 1 KM.,4600,2016/9819,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,MUJER |,HL |,DE 25 A 29 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 4600, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 SALAMANCA , ..."
86765,03/12/2010,DE 19:00 A 19:59,VIERNES,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,12000,2010/11139,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CAÃDA MOTOCICLETA |,MOTOCICLETA |,HOMBRE |,HL |,DE 21 A 24 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 12000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."
86769,11/11/2014,DE 10:00 A 10:59,MARTES,FUENCARRAL-EL PARDO,AUTOVIA M-30 CALZADA 2 KM.,32000,2014/10304,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32000, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 FUENCARRAL-EL PARDO , ..."
86795,23/05/2010,DE 6:00 A 6:59,DOMINGO,ARGANZUELA,AUTOVIA M-30 CALZADA 1 KM.,14200,2010/4575,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR |,CHOQUE CON OBJETO FIJO |,TURISMO |,MUJER |,HL |,DE 21 A 24 AÃOS |,"AUTOVIA M-30 CALZADA 1 KM. 14200, Madrid, Spain",NaN,NaN,NaN,NaN,"AUTOVIA M-30 ARGANZUELA , ..."


In [315]:
# Nos quedamos con las filas que contienen Autovía
dfAu = df[df['Autovia'].str.contains('AUTOVIA')]

In [316]:
# Y nos quedamos con los valores únicos para facilitar el scraping.
dfAu = pd.DataFrame(dfAu['Autovia'].unique()).astype(str)

In [317]:
dfAu = dfAu.rename(columns={0:'Autovia'})
dfAu

,Autovia
0,"AUTOVIA M-30 CHAMARTIN , ..."
1,"AUTOVIA M-30 ARGANZUELA , ..."
2,"AUTOVIA M-30 MORATALAZ , ..."
3,"AUTOVIA M-30 MONCLOA-ARAVACA , ..."
4,"AUTOVIA M-23 MORATALAZ , ..."
...,...
343,ACCESO A LA AUTOVIA M-40 - GLORIETA DE PRAGA H...
344,AUTOVIA M-30 - CALLE DE O'DONNELL CIUDAD LINE...
345,AUTOVIA M-30 - CARRETERA DE COLMENAR VIEJO HO...
346,AUTOVIA M-30 - CALLE DEL DOCTOR RAMON CASTROV...


In [318]:
from tqdm import tqdm
tqdm.pandas()
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TheCrash")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
dfAu['location'] = dfAu['Autovia'].progress_apply(geocode)

dfAu['point'] = dfAu['location'].apply(lambda loc: tuple(loc.point) if loc else None)

C:\Users\migue\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 348/348 [05:47<00:00,  1.00it/s]


In [319]:
dfAu.to_csv('./df62018_Accidentalidad.csv',sep=';', decimal='.')

In [320]:
dfAu = pd.read_csv('./df62018_Accidentalidad.csv',sep=';', decimal='.')
dfAu.head()

,Unnamed: 0,Autovia,location,point
0,0,"AUTOVIA M-30 CHAMARTIN , ...","Autovía del Norte, Castilla, Chamartín, Madrid...","(40.4838733, -3.6735923, 0.0)"
1,1,"AUTOVIA M-30 ARGANZUELA , ...","Autovía de Toledo, Chopera, Arganzuela, Madrid...","(40.396602, -3.7033402, 0.0)"
2,2,"AUTOVIA M-30 MORATALAZ , ...","Autovía del Este (Avenida del Mediterráneo), F...","(40.406094, -3.66066, 0.0)"
3,3,"AUTOVIA M-30 MONCLOA-ARAVACA , ...","Autovía del Noroeste, Aravaca, Moncloa-Aravaca...","(40.4609015, -3.7596365, 0.0)"
4,4,"AUTOVIA M-23 MORATALAZ , ...","Autovía del Este (Avenida del Mediterráneo), F...","(40.406094, -3.66066, 0.0)"


In [321]:
df['point3'] = df['Autovia'].map(dfAu.set_index('Autovia')['point'])

In [323]:
df['point'] = df['point'].fillna(df['point3'])

In [328]:
df.head(19)

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO PERSONA,TIPO ACCIDENTE,Tipo Vehiculo,SEXO,LESIVIDAD,Tramo Edad,Calle completa,point,CruceCalle,Calle1,point2,Autovia,point3
0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE |,MOTOCICLETA | TURISMO |,HOMBRE | HOMBRE |,HL | IL |,DE 21 A 24 AÃOS | DE 30 A 34 ANOS |,"AUTOVIA M-30 CALZADA 2 KM. 32300, Madrid, Spain","(40.4838733, -3.6735923, 0.0)",NaN,NaN,NaN,"AUTOVIA M-30 CHAMARTIN , ...","(40.4838733, -3.6735923, 0.0)"
1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,VIAJERO | CONDUCTOR | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | TURISMO | TURISMO | TURISMO | TURISM...,MUJER | HOMBRE | HOMBRE | MUJER | HOMBRE |,HL | IL | HL | HL | HL |,DE 18 A 20 AÃOS | DE 25 A 29 AÃOS | DE 30 A ...,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,"(40.4031169, -3.595555, 0.0)","[AVENIDA DE LA GRAN VIA DEL ESTE , CALLE DE L...","AVENIDA DE LA GRAN VIA DEL ESTE , Madrid, Spain","(40.4031169, -3.595555, 0.0)",AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,NaN
2,26/02/2012,DE 18:00 A 18:59,DOMINGO,ARGANZUELA,PASEO DE LAS DELICIAS - CALLE DE MURCIA,0,2012/2009,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CONDUCTOR | CONDUCTOR | CONDUCTOR | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,TURISMO | MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | NO ASIGNADO | NO ASIGNADO |,NO ASIGNADA | HL | NO ASIGNADA | NO ASIGNADA |,DE 18 A 20 AÃOS | DE 40 A 44 AÃOS | DESCONOC...,"PASEO DE LAS DELICIAS - CALLE DE MURCIA 0, Mad...","(40.3945911, -3.6946377, 0.0)","[PASEO DE LAS DELICIAS , CALLE DE MURCIA]","PASEO DE LAS DELICIAS , Madrid, Spain","(40.3945911, -3.6946377, 0.0)",PASEO DE LAS DELICIAS - CALLE DE MURCIA ARGANZ...,NaN
3,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,PEATON | PEATON | PEATON | PEATON | CONDUCTOR |,ATROPELLO | ATROPELLO | ATROPELLO | ATROPELLO ...,NO ASIGNADO | NO ASIGNADO | NO ASIGNADO | NO A...,HOMBRE | MUJER | MUJER | HOMBRE | HOMBRE |,HL | HL | HL | HL | IL |,DE 35 A 39 AÃOS | DE 35 A 39 AÃOS | DE 35 A ...,"CALLE DE EROS - CALLE DE LA ESTRELLA HADAR 0, ...","(40.3938219, -3.6802947, 0.0)","[CALLE DE EROS , CALLE DE LA ESTRELLA HADAR]","CALLE DE EROS , Madrid, Spain","(40.3938219, -3.6802947, 0.0)",CALLE DE EROS - CALLE DE LA ESTRELLA HADAR ARG...,NaN
4,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,CONDUCTOR | VIAJERO | CONDUCTOR |,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,MOTOCICLETA | TURISMO | TURISMO |,HOMBRE | HOMBRE | HOMBRE |,HL | IL | IL |,DE 30 A 34 ANOS | DE 18 A 20 AÃOS | DE 21 A 2...,"CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO 0, ...","(40.375867, -3.739077, 0.0)","[CALLE DE BELZUNEGUI , CALLE DE CARCASTILLO]","CALLE DE BELZUNEGUI , Madrid, Spain","(40.375867, -3.739077, 0.0)",CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO CAR...,NaN
5,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,CONDUCTOR | VIAJERO | VIAJERO | CONDUCTOR | VI...,COLISIÃN DOBLE | COLISIÃN DOBLE | COLISIÃN ...,FURGONETA | FURGONETA | FURGONETA | TURISMO | ...,HOMBRE | MUJER | MUJER | MUJER | MUJER |,IL | IL | HL | HL | HL |,DE 21 A 24 AÃOS | DE 0 A 5 AÃOS | DE 21 A 24...,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40...,"(40.4129832, -3.6559716, 0.0)","[AVENIDA DEL DOCTOR GARCIA TAPIA , AUTOVIA M,...","AVENIDA DEL DOCTOR GARCIA TAPI

In [329]:
df.to_csv('./dfsemifinal_Accidentalidad.csv',sep=';', decimal='.')

In [231]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="The Crash")
location2 = geolocator.geocode('AUTOVIA M-30, arganzuela, Madrid, Spain')
print(location2.address)
print((location2.latitude, location2.longitude))

Autovía de Toledo, Chopera, Arganzuela, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28001, España
(40.396602, -3.7033402)
